In [7]:
#决策树可用于回归或分类，并应用于多输出任务。
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier as DTC
iris=load_iris()
X=iris.data
y=iris.target
tree_clf=DTC(max_depth=4)
tree_clf.fit(X,y)
export_graphviz?
from sklearn.tree import export_graphviz
export_graphviz(
tree_clf,
out_file='iris_tree.dot',
feature_names=iris.feature_names,
class_names=iris.target_names,
rounded=True,
filled=True)

tree_clf.predict_proba([[5,4,3,2]])
tree_clf.predict([[3,4,5,6]])

from sklearn.tree import DecisionTreeRegressor as DTR
tree_reg=DTR(max_depth=4)
tree_reg.fit(X,y)


In [ ]:
#random forest
#voting 是利用不同的学习器对同一个数据集进行预测
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import VotingClassifier as VC
from sklearn.linear_model import LogisticRegression as Logit
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2)
voting_clf=VC(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],votting='hard')
votting_clf.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))
#soft voting,但需要每个基分类器都输出预测概率，而SVM则默认不输出概率，此时你需要将probabiliyty
#参数设为True。

#Bagging and Pasting
#Bagging是用同一个算法对数据集的不同子集进行预测，并且子集是有放回的。
#而Pasting是不放回的。
#bagging 和pasting都允许样本多次出现，但bagging允许样本出现在同一个分类器中。
#Bagging基于一些统计量如众数（类似于hard voting）进行最终的预测，或者分类结果的均值
#可进行并行计算
from sklearn.ensemble import BaggingClassifier as Bagging
from sklearn.tree import DecisionTreeClassifier ad DTC
bag_clf=Bagging(DTC(),n_estimators=500,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf.fit(X_train,y_train)
y_pred=bag_clf.predict(X_test)
#若基学习器可以预测概率，则BaggingClassifier自动进行软投票
#bagging比pasting的误差大但方差小,表现也更好，因此更受偏好。
#bagging中，平均每个回归器只利用了63%的样本（1-exp(-1）),其余37%的样本没有被抽到，称之为
#包外样本（out-of-bag）。因此这些样本可以自动用来进行单分类器的模型评价，而无需额外进行交叉验证。
#而单分类器模型的评分结果的均值又可以用来评价整个集成模型的效果。

#oob_score=True参数可以自动进行oob模型评价。
bag_clf=BaggingClassifier(DecisioTreeClassifier(),n_estimators=500,bootstrap=True,oob_score=True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)
#每个样本的oob决策函数由oob_decision_function_获得，由于基学习器可返回预测概率，所有该方法返回每个样本属于各类的概率。
bag_clf.oob_decision_function

##随机补丁和随机子空间
#BaggingClassifier支持对特征进行抽样，该方法由max_features 和bootstrap_features参数控制，
#其功能类似于max_samples和bootstrap，但不作用于样本而作用于特征。
#该功能在应对高维数据时非常有效，同时进行样本抽样和特征抽样的方法被称为Random Pathces方法
#只进行特征抽样（bootstrap_features=True,and/or max_features<1)而进行全样本抽样(bootstrap=Ture,
#max_samples=1.0)的方法被称为随机子空间方法。随机子空间法不属于Bagging方法。

#对特征进行抽样会带来更多的回归器差异，从而使误差变大而方差更小。

##随机森林
#随机森林是针对决策树的集成方法，本质上属于random patches，即在随机采集样本的基础上进一步对
#特征进行采样。
from sklearn.ensemble import RandomForestClassifier as RFC
rnd_clf=RFC(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred_rf=rnd_clf.predict(X_test)

#RF在进行决策树分枝特征选择时，并不是每次都在全局进行最优特征选取，而是在全局特征的一个子空间
#内进行最优特征选取，这虽然增加了偏差但降低了泛化误差，从而带来了更好表现。
bag_clf=Bagging(DTC(splitter='random',max_lef_nodes=16),n_estimators=500,max_samples=1,
                bootstrap=True,n_jobs=-1)


##Extra-Tree or Extremely random tree

#完全随机的得到分叉阈值，从而实现对决策树进行分叉的，而不是在子特征空间内寻找最优特征。

#通过计算在森林所有树中的平均深度来测度特征的相对重要性。
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RFC(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris['data'],iris['target'])
for name,score in zip(iris['feature_names'],rnd_clf.feature_importance_):#按原始顺序给出
    print(name,score)

In [5]:
##Boosting
#与Bagging不同，Boosting 方法是串行集成学习的代表，它以序贯的方式对基学习器进行训练，
#每次训练完一个基学习器后，利用Boosting方法以提升下一个学习器的表现，直至最后一个学习器。
#目前最流行的Boosting算法包括AdaBoost和Gradient Boosting.


##AdaBoost
#AdaBoost的基本思想是在前一个基分类器完成训练后，下之后的基分类器对前一个分类器欠拟合的样本
#赋予更高的权重，从而实现对欠拟合样本更好的拟合以提升后续分类器的表现，直至表现不再提升。
#预测权重最高的分类器将胜出。
#sklearn事实上用的是AdaBoost的多分类版本称为SAMME(Stagewise Additive Modeling using a 
# Multiclass Exponential loss function),当只有两类则等同于AdaBoost，但可以得的预测概率，
#sklearn可以调用SAMME的变体称为SAMME.R该算法依赖于类概率而不是预测。
#AdaBoosting算法的基学习器称为决策桩，即最大深度为1的决策树。

from sklearn.ensemble import AdaBoostClassifier as Ada
ada_clf=Ada(DTC(max_depth=1),n_estimators=200,algorithm='SAMME.R',learning_rate=0.05)
ada_clf.fit(X_train,y_train)


##Gradient Boosting
#与AdaBoosting不同之处在于，GBM是以新的学习器去拟合上一个学习器的残差。
from sklearn.tree import DecisionTreeRegressor as DTR
tree_reg1=DTR(max_depth=2)
tree_reg1.fit(X,y)
y2=y-tree_reg1.predict(X)
tree_reg2=DTR(max_depth=2)
tree_reg2.fit(X,yy2)
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

y_pred=sum(tree.predict(X_new)) for tree in (tree_reg1,tree_reg2,tree_reg3)
#GBR的大多数参数都是控制决策树的，如learing_rate控制每棵树的相对贡献。当learning_rate较小
#的情况下，树的规模必须很大，但模型的表现也越好。
#但树的最优规模仍需一些技巧才能确定，通常的方法是早停，staged_predict方法。
from sklearn.ensemble import GradientBoostingRegressor as GBR
gbrt=GBR(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X,y)

#从120开始，以每个阶段的验证误差为标准，确定树的最优规模，最后以最优规模训练一个完整的GBRT
#staged_predcit()方法返回一个迭代器，这个迭代器由集成器在每个阶段（从1迭代增加树的规模）
#的预测结果构成
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
X_train，X_val,y_train,y_val=train_test_split
gbrt=GBR(max_depth=2,n_estimators=120)
errors=[mse(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
best_n_estimators=np.argmin(errors)

#早停机制
gbrt=GBR(max_depth=2,warm_start=True)
min_val_error=float('inf')
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train,y_train)
    y_pred=gbrt.predict(X_val)
    val_error=mse(y_val,y_pred)
    if val_error<min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break
#GBR支持subsample参数，即指定各决策树用于训练的比例，如取0.25，那么每棵决策树只
#只随机选择25%的样本来训练，称之为Stochastic Gradient Boosting.

#loss 控制损失函数的选取。



In [ ]:
#Stacking.
##与其用简单函数来聚合各基学习器的预测结果，为什么不将所有基学习器进行堆栈以
#训练一个完整的模型呢。所谓堆栈即将前若干个基学习器的结果作为输入，以训练下一个学习器。
#最终的学习器称为blender以进行预测。
#训练blender的一种常用方法被称为hold-out set.即将训练集分为两个子集，第一个子集用来训练第一层
#并根据训练的的学习器预测第二个集合，这保证了每个学习器的预测是干净的，将预测结果作为输入，
#特征，并保留标签，并以最终融合器去进行预测。